# Part 3


The module 1 lecture example  didn't use langchain for orchestration.


Tasks:

- Basic prompting (What is the capital of France)
- Summarization - Summarizing an example text
- Information extraction - Getting information from a prompt
- Transformation - Translating a text, for example.
- Expansion or generative - Expanding a prompt into a more detailed response. e.g. Write a short story about the red planet, mars.
- Role based prompting - Instructing a model to respond in a specific persona
- Few shot prompting - giving the model examples int he prompt.
- chain of thought prompting - encouraging the model to reason out its steps.
- system prompts - setting out the behavior and the role of the ai model before user interaction
- utilized prompt - give the llm an example of both contexts: system, user assistant.

In [2]:
# Create the conda environment with a default python
#! conda create -n hw-1-submission python=3.11

# activate the conda environment
#! conda activate myenv 

# make sure you have a conda envornment, and have added the conda-forge channel.
#! conda install --name hw-1-submission  openai 

## AI Agent

Uses:

- Ollama (local) - smollm:1.7b because it is lightweight.
- Open AI Python Library
- Has two tools: add_numbers and subtract_numbers
- 

In [ ]:
import openai
import json

# Initialize the OpenAI client
client = openai.OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

# Define available functions
def add_numbers(a, b):
    return a + b

def subtract_numbers(a, b):
    return a - b

# Function to get the model's response
def get_agent_response(user_prompt, model="smollm2:1.7b"):
    messages = [{"role": "user", "content": user_prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        functions=[
            {
                "name": "add_numbers",
                "description": "Add two numbers",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {"type": "number", "description": "The first number"},
                        "b": {"type": "number", "description": "The second number"}
                    },
                    "required": ["a", "b"]
                }
            },
            {
                "name": "subtract_numbers",
                "description": "Subtract two numbers",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {"type": "number", "description": "The first number"},
                        "b": {"type": "number", "description": "The second number"}
                    },
                    "required": ["a", "b"]
                }
            }
        ],
        temperature=0,
    )

    response_message = response.choices[0].message

    if response_message.function_call:
        function_name = response_message.function_call.name
        arguments = json.loads(response_message.function_call.arguments)
        if function_name == "add_numbers":
            result = add_numbers(**arguments)
        elif function_name == "subtract_numbers":
            result = subtract_numbers(**arguments)
        else:
            result = "Function not recognized."
        return result
    else:
        return response_message.content

# Example usage
user_prompt = "What is 15 minus 7?"
response = get_agent_response(user_prompt)
print(response)


The answer is 6.


In [8]:
## Lets use langchain
! conda install langchain -y

2 channel Terms of Service accepted
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.9.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/iankt/anaconda3

  added / updated specs:
    - langchain


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    async-timeout-4.0.3        |     pyhd8ed1ab_0          11 KB  conda-forge
    conda-25.7.0               |  py313h78bf25f_0         1.2 MB  conda-forge
    langchain-1.0.2            |pymin313_h332efcf_0          86 KB  conda-forge
    langchain-core-1.0.1       |     pyhd8ed1ab_0         298 KB  conda-forge
    langchain-text-splitters-0.3.11|     pyhd8ed1ab_0          33 KB  conda-forge
    langgraph-1.0.1            |     pyhd8ed

In [ ]:
# create an agent
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
import time

# large language model
llm = ChatOllama(
    model="smollm2:1.7b",
    temperature=1
)

# tool to get weather information
def get_weather(city: str)->str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}"

def get_time(city: str)->str:
    """Gets the current local time"""
    current_time = time.strftime("%H:%M:%S", time.localtime())
    return f"Current time:{current_time}"


agent = create_agent(
    model=llm,
    tools=[get_weather, get_time],
    system_prompt="You are a helpful assistant."
)


# all the agent
results = agent.invoke(
    {"messages": [
        {
            "role": "user",
            "content":"what is the weather in Nairobi?"
        }
    ]}
)

for r in results["messages"]:
    print(f"{r}")

content='what is the weather in Nairobi?' additional_kwargs={} response_metadata={} id='b9b7151c-dcd5-4fb1-aa61-e6c80881a01d'
content='' additional_kwargs={} response_metadata={'model': 'smollm2:1.7b', 'created_at': '2025-10-29T04:13:14.023824809Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1812561362, 'load_duration': 43598220, 'prompt_eval_count': 316, 'prompt_eval_duration': 80669311, 'eval_count': 32, 'eval_duration': 1667989691, 'model_name': 'smollm2:1.7b', 'model_provider': 'ollama'} id='lc_run--13e4ccc5-4588-48e7-9a39-ae0e95bb7825-0' tool_calls=[{'name': 'get_weather', 'args': {'city': 'Nairobi'}, 'id': '12a7ec49-676e-4b03-bd23-0df39b5f5a28', 'type': 'tool_call'}] usage_metadata={'input_tokens': 316, 'output_tokens': 32, 'total_tokens': 348}
content="It's always sunny in Nairobi" name='get_weather' id='62dff961-c6f0-4fce-96a5-c050fcea4ac9' tool_call_id='12a7ec49-676e-4b03-bd23-0df39b5f5a28'
content='The output should not be provided as it does not match the format 

In [33]:
results = agent.invoke(
    {"messages" :[
        {
            "role":"user",
            "content":"What is the current time there?"}
    ]}
)
for r in results["messages"]:
    print(f"{r}")

content='What is the current time there?' additional_kwargs={} response_metadata={} id='bd0a02eb-0354-45f9-ba39-2b1eaffbffab'
content='' additional_kwargs={} response_metadata={'model': 'smollm2:1.7b', 'created_at': '2025-10-29T04:13:49.409323215Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1806466308, 'load_duration': 89317852, 'prompt_eval_count': 316, 'prompt_eval_duration': 456107400, 'eval_count': 25, 'eval_duration': 1239799059, 'model_name': 'smollm2:1.7b', 'model_provider': 'ollama'} id='lc_run--b82a85f1-09ba-4418-81ed-fffb60dcdd98-0' tool_calls=[{'name': 'get_time', 'args': {}, 'id': '673679af-758f-4c5b-9ac1-51acf087a62a', 'type': 'tool_call'}] usage_metadata={'input_tokens': 316, 'output_tokens': 25, 'total_tokens': 341}
content='Current time:21:13:49' name='get_time' id='f947183f-9eb2-4dc9-a3d8-1822dd0c7a18' tool_call_id='673679af-758f-4c5b-9ac1-51acf087a62a'
content='The output contains a tool response with the current time in 24-hour format.' additional_kwargs

In [34]:
llm.invoke("What is the capital city of Ethiopia? What is the time and weather there?")

AIMessage(content="The capital city of Ethiopia is Addis Ababa. Currently, it's 7:30 AM EST. As for the weather, it typically enjoys a temperate climate with average temperatures ranging from 55°F (13°C) to 68°F (20°C), depending on whether it's in the northern or southern parts of Ethiopia.", additional_kwargs={}, response_metadata={'model': 'smollm2:1.7b', 'created_at': '2025-10-29T04:38:43.827616611Z', 'done': True, 'done_reason': 'stop', 'total_duration': 14230994839, 'load_duration': 9595503548, 'prompt_eval_count': 45, 'prompt_eval_duration': 835114168, 'eval_count': 77, 'eval_duration': 3735432755, 'model_name': 'smollm2:1.7b', 'model_provider': 'ollama'}, id='lc_run--56b52755-3fc8-42d1-9869-25761a960a74-0', usage_metadata={'input_tokens': 45, 'output_tokens': 77, 'total_tokens': 122})

In [35]:
results = agent.invoke(
    {"messages" :[
        {
            "role":"user",
            "content":"What is the capital city of Ethiopia? What is the time and weather there?"}
    ]}
)
for r in results["messages"]:
    print(f"{r}")

content='What is the capital city of Ethiopia? What is the time and weather there?' additional_kwargs={} response_metadata={} id='7d7fff9e-d79c-4977-9d09-1a3cdb1ff8c7'
content='' additional_kwargs={} response_metadata={'model': 'smollm2:1.7b', 'created_at': '2025-10-29T04:40:59.94781555Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7041104435, 'load_duration': 106885253, 'prompt_eval_count': 325, 'prompt_eval_duration': 4618059238, 'eval_count': 46, 'eval_duration': 2269557249, 'model_name': 'smollm2:1.7b', 'model_provider': 'ollama'} id='lc_run--a73863f2-7103-42be-8aee-0bf197e5ffe2-0' tool_calls=[{'name': 'get_weather', 'args': {'city': 'Addis Ababa'}, 'id': '199ed493-4820-45a0-96ab-04c64671034a', 'type': 'tool_call'}, {'name': 'get_time', 'args': {}, 'id': 'f7238ec1-6fd0-42c4-9975-8c1c2bd33bfd', 'type': 'tool_call'}] usage_metadata={'input_tokens': 325, 'output_tokens': 46, 'total_tokens': 371}
content="It's always sunny in Addis Ababa" name='get_weather' id='9563ef5d-8a9